# Module 1

In [11]:
from utilities.ingest import ETL

e = ETL()

e.main()

0it [00:00, ?it/s]


ValueError: No objects to concatenate

In [3]:
import pandas as pd
import utilities.ingest as u


e = u.ETL(stock_id = "")
sample_sub = pd.read_csv('./data/sample_submission.csv')
book_train, trade_train, book_test, trade_test = e.ingest_all_parquet()

sample_sub.head()


In [4]:

book_train.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,stock_id
0,5,0,1.001422,1.002301,1.00137,1.002353,3,226,2,100,0
1,5,1,1.001422,1.002301,1.00137,1.002353,3,100,2,100,0
2,5,5,1.001422,1.002301,1.00137,1.002405,3,100,2,100,0
3,5,6,1.001422,1.002301,1.00137,1.002405,3,126,2,100,0
4,5,7,1.001422,1.002301,1.00137,1.002405,3,126,2,100,0


In [2]:
trade_train.head()

NameError: name 'trade_train' is not defined

In [7]:
pd.merge(book_train, trade_train, on=["time_id", "seconds_in_bucket"])

In [2]:

# Applying feature engineering to get feature engineering artifact
from utilities.feature_engineering import FeatureEngineering

fe = FeatureEngineering()
merged_df = fe.merge(curr_book = book_train, curr_trade = trade_train)

NameError: name 'book_train' is not defined

In [1]:


second_to_first_ratio(book_train)

NameError: name 'second_to_first_ratio' is not defined